## SETUP

In [1]:
from setup import setup

In [2]:
snapshot = 127
resolution = 4096
sim_id = "09_18"

In [3]:
(
    ds,
    mw,
    stellar_model,
    imf,
    planet_model,
    gas_projection_dict,
    particle_projection_dict,
) = setup(snapshot, resolution, sim_id)


      DETECTED LOCAL MACHINE: Test snapshot loaded.

yt : [INFO     ] 2023-06-05 12:14:04,417 Calculating time from 1.000e+00 to be 4.318e+17 seconds
yt : [INFO     ] 2023-06-05 12:14:04,485 Parameters: current_time              = 4.318436660023809e+17 s
yt : [INFO     ] 2023-06-05 12:14:04,486 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2023-06-05 12:14:04,487 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-06-05 12:14:04,487 Parameters: domain_right_edge         = [100. 100. 100.]
yt : [INFO     ] 2023-06-05 12:14:04,488 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-06-05 12:14:04,488 Parameters: current_redshift          = 4.440892098500626e-16
yt : [INFO     ] 2023-06-05 12:14:04,488 Parameters: omega_lambda              = 0.682
yt : [INFO     ] 2023-06-05 12:14:04,489 Parameters: omega_matter              = 0.318
yt : [INFO     ] 2023-06-05 12:14:04,489 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 202

## Metallicity

### Maps

In [4]:
# gas density-weighted gas metallicity
plot = yt.ProjectionPlot(
    fields=("gas", "metallicity"),
    weight_field=("gas", "density"),
    **gas_projection_dict,
)

plot.set_unit(("gas", "metallicity"), "Zsun")
plot.set_log(("gas", "metallicity"), False)

NameError: name 'yt' is not defined

In [ ]:
# average stellar metallicities
plot = yt.ParticleProjectionPlot(
    fields=("stars", "metallicity"),
    weight_field=("stars", "Masses"),
    **particle_projection_dict,
)
plot.set_unit(("stars", "metallicity"), "Zsun")
plot.set_log(("stars", "metallicity"), False)
plot.set_zlim(("stars", "metallicity"), zmin=(0, "Zsun"), zmax=(6, "Zsun"))

### Profiles

In [ ]:
# density-weighted gas metallicity
gas_profile = yt.create_profile(
    data_source=mw.disk(radius=ds.quan(20, "kpc")),
    bin_fields=[("gas", "radius")],
    fields=[("gas", "metallicity")],
    n_bins=100,
    units={("gas", "radius"): "kpc", ("gas", "metallicity"): "Zsun"},
    weight_field=("gas", "density"),
)

plot = yt.ProfilePlot.from_profiles(gas_profile)
plot.set_log(("gas", "radius"), False)
plot.set_log(("gas", "metallicity"), False)
plot.set_ylim(("gas", "metallicity"), 0.5, 6)

In [ ]:
# average stellar metallicity profile
star_profile = yt.create_profile(
    data_source=mw.disk(radius=ds.quan(20, "kpc")),
    bin_fields=[("stars", "particle_radius")],
    fields=[("stars", "metallicity")],
    n_bins=100,
    units={("stars", "particle_radius"): "kpc", ("stars", "metallicity"): "Zsun"},
    weight_field=("stars", "Masses"),
)

plot = yt.ProfilePlot.from_profiles(star_profile)
plot.set_log(("stars", "particle_radius"), False)
plot.set_log(("stars", "metallicity"), False)
plot.set_ylim(("stars", "metallicity"), 0.5, 3)

In [ ]:
# metallicity histogram - galactic center
fig, ax = plt.subplots(figsize=(10, 8))
ax.hist(
    mw.sphere(radius=ds.quan(0.5, "kpc"))["gas", "metallicity"].to("Zsun").value,
    density=True,
    bins="fd",
    label="gas",
    alpha=0.5,
)
ax.hist(
    mw.sphere(radius=ds.quan(0.5, "kpc"))["stars", "metallicity"].to("Zsun").value,
    density=True,
    bins="fd",
    label="stars",
    alpha=0.5,
)
ax.set_xlabel("Metallicity in solar metallicities")
ax.set_ylabel("Normalized Count")
ax.set_title("Metallicity in the galactic center (r = 0.5 kpc)")
ax.legend()

In [ ]:
# metallicity histogram - 20kpc
fig, ax = plt.subplots(figsize=(10, 8))
ax.hist(
    mw.sphere(radius=ds.quan(20, "kpc"))["gas", "metallicity"].to("Zsun").value,
    density=True,
    bins="fd",
    label="gas",
    alpha=0.5,
)
ax.hist(
    mw.sphere(radius=ds.quan(20, "kpc"))["stars", "metallicity"].to("Zsun").value,
    density=True,
    bins="fd",
    label="stars",
    alpha=0.5,
)
ax.set_xlabel("Metallicity in solar metallicities")
ax.set_ylabel("Normalized Count")
ax.set_title("Metallicity in the galaxy (r = 20 kpc)")
ax.legend()

## Stellar Age

In [ ]:
# average stellar metallicities
plot = yt.ParticleProjectionPlot(
    fields=("stars", "stellar_age"),
    weight_field=("stars", "Masses"),
    **particle_projection_dict,
)
# plot.set_unit(("PartType4", "stellar_age"), "Gyr")
plot.set_log(("stars", "stellar_age"), False)
# plot.set_zlim(("PartType4", "metallicity"), zmin=(0, "Zsun"), zmax=(6, "Zsun"))

In [ ]:
# average stellar age profile
star_profile = yt.create_profile(
    data_source=mw.disk(radius=ds.quan(20, "kpc")),
    bin_fields=[("stars", "particle_radius")],
    fields=[("stars", "stellar_age")],
    n_bins=100,
    units={("stars", "particle_radius"): "kpc", ("stars", "stellar_age"): "Gyr"},
    weight_field=("stars", "Masses"),
)

plot = yt.ProfilePlot.from_profiles(star_profile)
plot.set_log(("stars", "particle_radius"), False)
plot.set_log(("stars", "stellar_age"), False)

## Planets

In [ ]:
# planet profile
planet_profile = yt.create_profile(
    data_source=mw.disk(radius=ds.quan(20, "kpc")),
    bin_fields=[("stars", "particle_radius")],
    fields=[("stars", "planets")],
    n_bins=100,
    units={("stars", "particle_radius"): "kpc"},
    weight_field=None,
)


# plot = yt.ProfilePlot.from_profiles(planet_profile)
# plot.set_log(('stars', 'particle_radius'), False)
# plot.set_log(('stars','planets'), False)
# plot.set_ylim(('stars','metallicity'),0.5, 3)
# show_or_save(plot, mode)

In [ ]:
# mass-weighted planet profile
planet_profile = yt.create_profile(
    data_source=mw.disk(radius=ds.quan(20, "kpc")),
    bin_fields=[("stars", "particle_radius")],
    fields=[("stars", "planets")],
    n_bins=100,
    units={("stars", "particle_radius"): "kpc"},
    weight_field=("stars", "Masses"),
)

plot = yt.ProfilePlot.from_profiles(planet_profile)
plot.set_log(("stars", "particle_radius"), False)
plot.set_log(("stars", "planets"), False)
show_or_save(plot, mode)

In [ ]:
plot = yt.ParticleProjectionPlot(
    fields=("stars", "planets"),
    weight_field=("stars", "main_sequence_stars"),
    **particle_projection_dict,
)

In [ ]:
plot.set_log(("stars", "planets"), False)
plot.set_zlim(("stars", "planets"), zmin=(10000, ""), zmax=(300000, ""))
plot.show()

In [ ]:
def _av_p(field, data):
    return data["stars", "planets"] / data["stars", "main_sequence_stars"]


ds.add_field(("stars", "av_p"), function=_av_p, sampling_type="local", units="")

In [ ]:
plot = yt.ParticleProjectionPlot(
    fields=("stars", "av_p"),
    weight_field=("stars", "particle_ones"),
    **particle_projection_dict,
)

In [ ]:
plot.set_log(("stars", "av_p"), False)
plot.set_zlim(("stars", "av_p"), zmin=(0.3, ""), zmax=(0.5, ""))
plot.show()